In [21]:
class Customer:
    def __init__(self, arrive_time, quantity, service_time, x, y):
        self.arrive_time = arrive_time
        self.quantity = quantity
        self.service = service_time
        self.x = x
        self.y = y


class Truck_Solution:
    def __init__(self, assigned_customers, recived_truck, recived_drone, arrive_time, leave_time):
        self.assigned_customers = assigned_customers
        self.recived_truck = recived_truck
        self.recived_drone = recived_drone
        self.arrive_time = arrive_time
        self.leave_time = leave_time

class Drone_Trip:
    def __init__(self, assigned_customers, recived_drone, arrive_time, leave_time):
        self.assigned_customers = assigned_customers
        self.recived_drone = recived_drone
        self.arrive_time = arrive_time
        self.leave_time = leave_time

class Drone_Solution:
    def __init__(self, num_of_trips, trip_list):
        self.num_of_trips = num_of_trips
        self.trip_list = trip_list

        
class Problem:
    def __init__(self, customer_list, number_of_trucks, number_of_drones, distance_matrix_truck,
                distance_matrix_drone, truck_capacity, drone_capacity, drone_energy,
                speed_of_truck, speed_of_drone, launch_time, land_time,
                energy_consumption_rate, weight_of_drone):
        self.customer_list = customer_list
        self.number_of_trucks = number_of_trucks
        self.number_of_drones = number_of_drones
        self.distance_matrix_truck = distance_matrix_truck
        self.distance_matrix_drone = distance_matrix_drone
        self.truck_capacity = truck_capacity
        self.drone_capacity = drone_capacity
        self.drone_energy = drone_energy
        self.speed_of_truck = speed_of_truck
        self.speed_of_drone = speed_of_drone
        self.launch_time = launch_time
        self.land_time = land_time
        self.energy_consumption_rate = energy_consumption_rate
        self.weight_of_drone = weight_of_drone

    def check_truck_capacity(self, route):
        total = 0
        for customer in route:
            total = total + customer[1]
        if total <= self.Q:
            return True
        else:
            return False

    def check_drone_capacity(self, route):
        total = 0
        for customer in route:
            total = total + customer[1]
        if total <= self.tilde_Q:
            return True
        else:
            return False

        
    def cal_truck_route_time(self, route):
        total_time = 0
        if len(route) == 0:
            return 0
        total_time = total_time + self.d[0][route[0][0]]/self.v + self.customer_list[route[0][0]].t
        for i in range(len(route) - 1):
            total_time = total_time + self.d[route[i][0]][route[i+1][0]]/self.v + self.customer_list[route[i+1][0]].t
        total_time = total_time + self.d[route[-1][0]][0]/self.v
        return total_time
    
    def check_truck_time_constraint(self, route):
        total_time = self.cal_truck_route_time(route)
        if total_time <= self.T:
            return True
        else:
            return False

    def cal_drone_route_time(self, route):
        total_time = 0
        if len(route) == 0:
            return 0
        total_time = total_time + self.tau_l + self.tilde_d[0][route[0][0]]/self.tilde_v + self.tau_r + self.customer_list[route[0][0]].t
        for i in range(len(route) - 1):
            total_time = total_time + self.tau_l + self.tilde_d[route[i][0]][route[i+1][0]]/self.tilde_v + self.tau_r + self.customer_list[route[i+1][0]].t
        total_time = total_time + self.tau_l + self.tilde_d[route[-1][0]][0]/self.tilde_v + self.tau_r
        return total_time
    
    def check_drone_time_constraint(self, multi_route):
        total_time = 0
        for route in multi_route:
            total_time = total_time + self.cal_drone_route_time(route)
        if total_time <= self.T:
            return True
        else:
            return False

    def cal_drone_route_energy(self, route):
        total_energy = 0
        if len(route) == 0:
            return 0
        total_demand = 0
        for customer in route:
            total_demand = total_demand + customer[1]

        total_energy = total_energy + self.lbda*(self.tilde_qo + total_demand)*self.tilde_d[0][route[0][0]]/self.tilde_v 
        total_demand = total_demand - route[0][1]
        total_energy = total_energy + self.lbda* total_demand * self.customer_list[route[0][0]].t
        for i in range(len(route) - 1):
            total_energy = total_energy + self.lbda*(self.tilde_qo + total_demand)*self.tilde_d[route[i][0]][route[i+1][0]]/self.tilde_v
            total_demand = total_demand - route[i+1][1]
            total_energy = total_energy + self.lbda* total_demand * self.customer_list[route[i+1][0]].t

        total_energy = total_energy + self.lbda*(self.tilde_qo + total_demand)*self.tilde_d[route[-1][0]][0]/self.tilde_v 
        return total_energy
    
    def check_drone_energy_constraint(self, route):
        total_energy = self.cal_drone_route_energy(route)
        if total_energy <= self.E:
            return True
        else:
            return False
        



In [22]:
import sys
import os
src_path = r"./"
sys.path.append(src_path)

In [ ]:
class State:
    def __init__(self, problem):
        self.truck = []             # location of trucks
        self.drones = []            # location of drones
        self.time = 0

SyntaxError: incomplete input (655263051.py, line 1)

In [ ]:
from population import Individual

'''
[1, -1, 3, 7, 4, 0, 5, 6, -1, 8, 2]
[0,  0, 0, 1, 0, 0, 1, 0,  0, 1, 0]

0: truck
1: drone

0: truck split
-1: drone split
1 '0' means there are 2 trucks
2 '-1' means there are 3 drones

truck route: [1, 3, 4], [6, 2]
drone route: [], [7, 5], [8]

'''
def extract_routes(chromosome):
    truck_routes = []
    drone_routes = []
    tmp_truck_route = []
    tmp_drone_route = []
    
    for i in range(len(chromosome[0])):
        if chromosome[0][i] == 0:
            truck_routes.append(tmp_truck_route)
            tmp_truck_route = []
            continue
        elif chromosome[0][i] == -1:
            drone_routes.append(tmp_drone_route)
            tmp_drone_route = []
            continue
        
        if chromosome[1][i] == 0:
            tmp_truck_route.append(chromosome[0][i])
        elif chromosome[1][i] == 1:
            tmp_drone_route.append(chromosome[0][i])
    
    truck_routes.append(tmp_truck_route)
    drone_routes.append(tmp_drone_route)
    print("Truck routes:", truck_routes)
    print("Drone routes:", drone_routes)
    
    return truck_routes, drone_routes


def decode(individual: Individual, problem: Problem):
    chromosome = individual.chromosome
    truck_routes, drone_routes = extract_routes(chromosome)
    
    # sort delivery sequence by arrival time
    for route in truck_routes:
        route.sort(key=lambda cust: problem.customer_list[cust].arrive_time)
    
    for route in drone_routes:
        route.sort(key=lambda cust: problem.customer_list[cust].arrive_time)
        
    if problem is None:
        return
    
    timestamps = [problem.customer_list[cust].arrive_time for cust in problem.customer_list]
    timestamps.sort()    
    
    for t in timestamps:
        update_state()        
    # visited = [False for _ in range(len(problem.customer_list)+1)]
    # visited[0] = True
    # priority_queue = []
    
    # this loop will terminate when all customers are visited and all used drones are collected or return to depot
    while True:
        # select the lauching drone:
        #   choose the drone which has the node with lowest arrival time
        
        #   
        # select truck node the selected drone will launch from
        #   choose the closest one to the first node the drone will visit in its route
        #   + if the node is still infeasible, select the one with the second lowest arrival time
        
        # pseudocode:
        #   + iterating through the list of drone routes which is ordered by the first unvisited customer's arrival time
        #       ? how many customers should be picked up by the drone in this trip
        #       + iterating through the list of unvisited customers in the current drone route
        #           ? should be picked up by the drone in this trip
        #   + update the state of truck routes
        #   
        
        pass
    
            

In [26]:
chromosome = [[1, -1, 3, 7, 4, 0, 5, 6, -1, 8, 2],
              [0,  0, 0, 1, 0, 0, 1, 0,  0, 1, 0]]

indi = Individual(chromosome)
decode(indi, None)

Truck routes: [[1, 3, 4], [6, 2]]
Drone routes: [[], [7, 5], [8]]
